In [1]:
#Bouaoune Youghourta BOUY28059509 
import pandas as pd
import numpy as np
from scipy import linalg
from numpy.linalg import inv
from numpy.linalg import qr
import scipy.linalg as linalg
import matplotlib.pyplot as plt
import statistics
import math
from google.colab import files
data = files.upload()
df = pd.read_csv('salaires.csv')
df.columns
df

Saving salaires.csv to salaires.csv


,age,classe.travail,education,nombre.education,etat.civil,occupation,Lien.parente,race,sexe,heures.par.semaine,pays.natal,salaire
0,90,?,HS-grad,9,Veuf,?,Pas_dans_famille,Blanc,Feminin,40,United-States,<=50K
1,82,Privee,HS-grad,9,Veuf,Exec-managerial,Pas_dans_famille,Blanc,Feminin,18,United-States,<=50K
2,66,?,Some-college,10,Veuf,?,Celibataire,Noir,Feminin,40,United-States,<=50K
3,54,Privee,7th-8th,4,Divorce,Machine-op-inspct,Celibataire,Blanc,Feminin,40,United-States,<=50K
4,41,Privee,Some-college,10,Separe,Prof-specialty,Un_enfant,Blanc,Feminin,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Privee,Some-college,10,Jamais_marie,Protective-serv,Pas_dans_famille,Blanc,Masculin,40,United-States,<=50K
32557,27,Privee,Assoc-acdm,12,Marie_civil,Tech-support,Epouse,Blanc,Feminin,38,United-States,<=50K
32558,40,Privee,HS-grad,9,Marie_civil,Machine-op-inspct,Mari,Blanc,Masculin,40,United-States,>50K
32559,58,Privee,HS-grad,9,Veuf,Adm-clerical,Celibataire,Blanc,Feminin,40,United-States,<=50K


In [ ]:
###############################6.2###########################
#6.2 Exploration des donn´ees (3pts)
#1. Quel est la taille du jeu de donn´ees ?
#réponce 1):
taille_df = df.size
print('la taille de jeux de donnes',taille_df)

la taille de jeux de donnes 390732


In [ ]:
#2. Combien de valeurs manquantes y a-t-il pour chaque colonne ? Quelle est
#la colonne (feature) qui a le plus de valeurs manquantes ?

cols = ['age', 'classe.travail', 'education', 'nombre.education',
       'etat.civil', 'occupation', 'Lien.parente', 'race', 'sexe',
       'heures.par.semaine', 'pays.natal', 'salaire']
df[cols] = df[cols].replace({'?':np.nan, '!':np.nan})
print(df.isnull().sum())
max(df.isnull().sum())

age                      0
classe.travail        1836
education                0
nombre.education         0
etat.civil               0
occupation            1843
Lien.parente             0
race                     0
sexe                     0
heures.par.semaine       0
pays.natal             583
salaire                  0
dtype: int64


1843

In [2]:
#3) Combien d’hommes y a-t-il dans ce jeu de donn´ees ?
sexe_c = df['sexe']
nb_homme = sexe_c.loc[(df['sexe'] == 'Masculin')]
nb_homme
print(len(nb_homme))

21790


In [2]:
#4)Quel est l’ˆage le plus ´elev´e et le plus bas de ce jeu de donnees ?
age_max = max(df['age'])
print('age max',age_max)

age_min = min(df['age'])
print('age min ',age_min)


age max 90
age min  17


In [3]:
#5)Combien de femmes travaillent dans le secteur priv´e?
df_2 = df[['sexe','classe.travail']]

feminin_privee = df_2.loc[(df_2['sexe'] == 'Feminin') & (df_2['classe.travail'] == 'Privee')]
print(feminin_privee)
M = len(feminin_privee)
print('les femmes travaillent dans le secteur privée',M)

          sexe classe.travail
1      Feminin         Privee
3      Feminin         Privee
4      Feminin         Privee
5      Feminin         Privee
10     Feminin         Privee
...        ...            ...
32539  Feminin         Privee
32546  Feminin         Privee
32548  Feminin         Privee
32557  Feminin         Privee
32559  Feminin         Privee

[7752 rows x 2 columns]
les femmes travaillent dans le secteur privée 7752


In [4]:
#6)Quelle est la liste des diff´erents pays repr´esent´es dans la colonne repr´esentant
#le pays natal/d’origine de chaque personne de notre jeu de donn´ees?
countries = df['pays.natal'].drop_duplicates()
print(countries)

#print(df['pays.natal'].info)

0                    United-States
9                                ?
26                          Mexico
27                          Greece
47                         Vietnam
63                           China
82                          Taiwan
90                           India
95                     Philippines
156                Trinadad&Tobago
157                         Canada
166                          South
211             Holand-Netherlands
216                    Puerto-Rico
268                         Poland
284                           Iran
287                        England
412                        Germany
447                          Italy
487                          Japan
508                           Hong
535                       Honduras
726                           Cuba
781                        Ireland
921                       Cambodia
938                           Peru
954                      Nicaragua
1004            Dominican-Republic
1044                

In [ ]:
##################################################6.3######################################
#6.3 Alg`ebre lin´eaire (6pts)
df = pd.read_csv('salaires.csv')
#1)1. Peut-on pr´edire l’ˆage d’une personne (femme) en fonction de son niveau
#d’´education et son nombre d’heures travaill´ees par semaine ? Vous devez
#passer par la technique de r´esolution d’´equations lin´eaires par la factorisation
#QR. Vous devez tester votre mod`ele sur les donn´ees de tests.
df[['sexe','salaire']] = df[['sexe','salaire']].astype('category')
df = df.query('sexe == "Feminin"')

test_start = len(df)*9 //10

x_data_fit = df[['nombre.education','heures.par.semaine']][:test_start]
x_data_test = df[['nombre.education','heures.par.semaine']][test_start:]
Y = df['age'][:test_start]
y_test = df['age'][test_start:]

In [ ]:
# fit data (append ones to account for constant term)
X = np.append(x_data_fit,np.ones((test_start,1)), axis=1)

#test data
x_test = np.append(x_data_test, np.ones((len(df)-test_start,1)), axis=1)

# QR decomposition
Q, R = qr(X)
A = inv(R).dot(Q.T).dot(Y)
#print(A) #  X * A = Y 

# predict using coefficients
y_pred = x_test.dot(A)
print('mean absolute error (years):',abs((y_pred-y_test)).mean())

mean absolute error (years): 11.213539741482565


In [ ]:
#2) Si le niveau d’´education et le nombre d’heures travaill´ees ´etaient r´eduits `a
#1 seule dimension qui capture les informations importantes des 2 (SVD) ?
#est-ce que la pr´ediction serait am´elior´ee ou pas ? Vous devez tester votre
#mod`ele sur les donn´ees de tests.
U, s, Vh = linalg.svd(x_data_fit, check_finite=False, full_matrices = False)

# combine education and wotk hour into a single column
new_column = x_data_test.dot(Vh[0]) / s[0]
x_test = np.append(np.array(new_column)[np.newaxis].T, np.ones((len(df)-test_start,1)),axis=1)
# the first column of U is x_data_fit.dot(Vh[[0]]) / s[0]

X = np.append(U[:,[0]],np.ones((test_start,1)), axis=1)


# QR decomposition
Q, R = qr(X)
A =inv(R).dot(Q.T).dot(Y)
#print(A) 

# predict using coefficients
y_pred = x_test.dot(A)
print('mean absolute error (years):',abs((y_pred-y_test)).mean()) 

# we are reducing dimension (loosing some data) this should theoriticaly reduce precision 

mean absolute error (years): 11.234471291795218


In [ ]:
#3)Peut-on pr´edire si le salaire d’une femme sera ≤ 50K en fonction de son
#niveau d’´education, son ˆage et son nombre d’heures travaill´ees par semaine
#? Vous devez passer par la technique de r´esolution d’´equations lin´eaires.
#Remarque : puisque le Y `a pr´edire est binaire alors ce n’est normalement
#pas un probl`eme de r´egression lin´eaire. Vous devez donc adapter votre
#solution en cons´equence. Par exemple, si la pr´ediction est inf´erieure `a 0,5
#alors le salaire est ≤ 50k sinon c’est > 50k. Vous devez tester votre mod`ele
#sur les donn´ees de tests.
x_data_fit = df[['nombre.education','heures.par.semaine','age']][:test_start]
x_data_test = df[['nombre.education','heures.par.semaine','age']][test_start:]
Y = (df['salaire'][:test_start] == '>50K').astype('int')
y_test =(df['salaire'][test_start:] == '>50K').astype('int') 


# fit data (append ones to account for constant term)
X = np.append(x_data_fit,np.ones((test_start,1)), axis=1)

#test data
x_test = np.append(x_data_test, np.ones((len(df)-test_start,1)), axis=1)

# QR decomposition
Q, R = qr(X)
A =inv(R).dot(Q.T).dot(Y)
#print(A) #  X * A = Y 

# predict using coefficients
y_pred = (0.5 < x_test.dot(A)).astype('int')

print('Accuracy (%):',(y_pred == y_test).mean()*100)
# note:  50k$ was not a great treshold salaries under that values are over represented in the dataset

Accuracy (%): 91.09461966604823


In [4]:
##################################################6.4#####################################
#6.4 Analyses statistiques et probabilistes (12pts)
df = pd.read_csv('salaires.csv')
#1)Quelle est la moyenne des ˆages ? Quelle est la moyenne du nombre d’heures
#travaill´ees par semaine ? et les valeurs m´edianes ?
moy_age = df['age'].mean()
print('moy age=',moy_age)
med_age = df['age'].median()
print('median age=',med_age)

moy_heur = df['heures.par.semaine'].mean()
print('moyean heur=',moy_heur)
med_heur = df['heures.par.semaine'].median()
print('median heur=',med_heur)

moy age= 38.58164675532078
median age= 37.0
moyean heur= 40.437455852092995
median heur= 40.0


In [ ]:
#2)Quelle sont les ˆages (prendre les 5 premiers) les plus repr´esentatifs de ce
#jeu de donn´ees ? Repr´esentent-ils une tranche d’ˆage en particulier (ex:
#30-35 ans) ?
mode_age = df['age'].value_counts()
print(mode_age)
taille = mode_age[:5]
taille

36    898
31    888
34    886
23    877
35    876
     ... 
83      6
88      3
85      3
86      1
87      1
Name: age, Length: 73, dtype: int64


36    898
31    888
34    886
23    877
35    876
Name: age, dtype: int64

In [5]:
#3)Comment est la variance (`a quel point les donn´ees sont ´eparpill´es/dispers´ees)
#des heures travaill´ees par semaine ? Et les ˆages ?
var_heur = df['heures.par.semaine'].var()
print('la variance des heures.par.semaine ',var_heur)
ecat_type_heur = math.sqrt(var_heur)
print('ecat_type_heur=',ecat_type_heur)
var_age = df['age'].var()
ecat_type_age = math.sqrt(var_age)
print('la varinace des ages',var_age)
print('ecat_type_age =',ecat_type_age)


la variance des heures.par.semaine  152.45899505045415
ecat_type_heur= 12.347428681731843
la varinace des ages 186.06140024880162
ecat_type_age = 13.640432553581343


In [ ]:
#4)
#Question 4 : Est-ce que le niveau d’éducation, comparé à l’âge influe le plus sur la valeur du salaire ? En d’autres termes est que le niveau d’éducation a plus de poids sur le salaire que l’âge ??
#Réponse : Oui parce que la corrélation entre le niveau d’éducation et le salaire (0.335154) est supérieure à la corrélation entre l’âge et le salaire et le salaire (0.234037)
#cov1 = df['nombre.education'].cov(df['age'])
#print('la covariance d age et nombre.education:',cov1)
#Si la covariance est supérieure à 0, on peut dire qu'il ya corrélation entres les deux séries, mais on ne peut pas conclure qu'il y causalité
#df['salaire'] = (df['salaire']).replace(to_replace = ['>50K','<=50K'], value = [1,0])
#corr1 = (df['nombre.education']).corr(df['age'],method='pearson', min_periods=1)
#print('la corilation entr l age et nombre education :',corr1)
#cov1 = df.corr()
#print(cov1)
df['salaire'] = df['salaire'].replace(to_replace = ['>50K','<=50K'], value = [1,0])
corrMatrix = df.corr()
print (corrMatrix)

                         age  nombre.education  heures.par.semaine   salaire
age                 1.000000          0.036527            0.068756  0.234037
nombre.education    0.036527          1.000000            0.148123  0.335154
heures.par.semaine  0.068756          0.148123            1.000000  0.229689
salaire             0.234037          0.335154            0.229689  1.000000


In [ ]:
#5)Est ce que le niveau d’´education, compar´e au nombre d’heures travaill´e
#par semaine influe le plus sur la valeur du salaire ? En d’autres termes
#est que le niveau d’´education a plus de poids sur le salaire que le nombre
#d’heures travaill´ees par semaine ?

df['salaire'] = df['salaire'].replace(to_replace = ['>50K','<=50K'], value = [1,0])
corrMatrix = df.corr()
print (corrMatrix)

                         age  nombre.education  heures.par.semaine   salaire
age                 1.000000          0.036527            0.068756  0.234037
nombre.education    0.036527          1.000000            0.148123  0.335154
heures.par.semaine  0.068756          0.148123            1.000000  0.229689
salaire             0.234037          0.335154            0.229689  1.000000


In [8]:
#6)`A quel point l’ˆage est un facteur qui influe le salaire? Pour r´epondre `a
#cette question vous devez v´erifier que l’ˆage moyen de ceux qui gagne moins
#de 50k est plus bas que ceux qui gagnent plus de 50k.
age_M1 = (df.loc[df['salaire'] == '<=50K','age']).mean()
print('le nombre d age moyenne pour les salaries <=50k est :',age_M1)

age_M2 = (df.loc[df['salaire'] == '>50K','age']).mean()
print('le nombre d age moyenne pour les salaries >50k est :',age_M2)

comapre_age = age_M1 < age_M2
print('le nombre d age moyenne pour les salaries <=50k < le nombre d age moyenne pour les salaries >50k ==',comapre_age)

le nombre d age moyenne pour les salaries <=50k est : 36.78373786407767
le nombre d age moyenne pour les salaries >50k est : 44.24984058155847
le nombre d age moyenne pour les salaries <=50k < le nombre d age moyenne pour les salaries >50k == True


In [9]:
#7)`A quel point le nombre d’heures travaill´e par semaine est un facteur qui
#influe le salaire? Pour r´epondre `a cette question vous devez v´erifier que le
#nombre moyen d’heure travaill´e par semaine pour ceux qui gagnent moins
#de 50k est plus bas que ceux qui gagnent plus de 50k.'
heur_M1 = (df.loc[df['salaire'] == '<=50K','heures.par.semaine']).mean()
print('le heures.par.semaine moyenne pour les salaries <=50k est :',heur_M1)

heur_M2 = (df.loc[df['salaire'] == '>50K','heures.par.semaine']).mean()
print('le heures.par.semaine moyenne pour les salaries >50k est :',heur_M2)

comapre_heur = heur_M1 < heur_M2
print('le heures.par.semaine moyenne pour les salaries <=50k < le heures.par.semaine moyenne pour les salaries >50k ==',comapre_heur)

le heures.par.semaine moyenne pour les salaries <=50k est : 38.840210355987054
le heures.par.semaine moyenne pour les salaries >50k est : 45.473026399693914
le heures.par.semaine moyenne pour les salaries <=50k < le heures.par.semaine moyenne pour les salaries >50k == True


In [20]:
#8) Est ce qu’il y a une diff´erence de salaire entre les personnes mari´ees et les
#personnes c´elibataires ? Quel groupe gagne le mieux leurs vies ?
df_salaire = df['salaire']
df_etat_civil_marie_salaire =df_salaire[(df['etat.civil'] == 'Marie_civil') | (df['etat.civil'] == 'Marie_epoux_absent')]
print(df_etat_civil_marie_salaire)
print('salaire pour les personnes mariés = ',df_etat_civil_marie_salaire.count())
df_etat_civil_selibataire_salaire = df_salaire[(df['etat.civil'] == 'Veuf') | (df['etat.civil'] == 'Divorce')]
print('salaire pour les personnes cilibataire = ',df_etat_civil_selibataire_salaire.count())
if df_etat_civil_selibataire_salaire.count() != df_etat_civil_marie_salaire.count():
   print('les deux salaire sont diffirent')

23       <=50K
24       <=50K
25       <=50K
26       <=50K
27       <=50K
         ...  
32552    <=50K
32553    <=50K
32555     >50K
32557    <=50K
32558     >50K
Name: salaire, Length: 15394, dtype: object
salaire pour les personnes mariés =  15394
salaire pour les personnes cilibataire =  5436
les deux salaire sont diffirent


In [ ]:
#9)Quelle est la proportion des hommes qui ont un doctorat et gagnent plus
#de 50k ? et chez les femmes ?
df = pd.read_csv('salaires.csv')
df2 = df['education'].drop_duplicates()
#print(df2)
homme_prop = len(df.loc[(df['education'] == 'Doctorate') & (df['salaire'] == '>50K') & (df['sexe'] == 'Masculin')])
print('la taille de notre population :',homme_prop)
homme_prop
x =(df.loc[(df['sexe'] == 'Masculin')])
x = len(x)    
print('la taille totale de df : ',x)
homme_prop = (homme_prop*100)/x
print('le pourcentage des hommes ont NV doctorat et un salaire de >50K : %',homme_prop)

femme_prop = len(df.loc[(df['education'] == 'Doctorate') & (df['salaire'] == '>50K') & (df['sexe'] == 'Feminin')])
print('la taille de notre population :',femme_prop)
x =(df.loc[(df['sexe'] == 'Feminin')])
x = len(x)
print('la taille totale de df : ',x)
femme_prop = (femme_prop*100)/x
print('le pourcentage des femme ont NV doctorat et un salaire de >50K : %',femme_prop)

la taille de notre population : 256
la taille totale de df :  21790
le pourcentage des hommes ont NV doctorat et un salaire de >50K : % 1.174850849013309
la taille de notre population : 50
la taille totale de df :  10771
le pourcentage des femme ont NV doctorat et un salaire de >50K : % 0.46420945130442853


In [21]:
#10)Quelle est la proportion des femmes mari´ees qui n’ont pas fait de masters
#ni de doctorat ? et chez les hommes ?
femme_prop = len(df.loc[((df['etat.civil'] == 'Marie_epoux_absent') | (df['etat.civil'] == 'Marie_civil')) &  ((df['education'] != 'Doctorate') & (df['education'] != 'Masters') & (df['sexe'] == 'Feminin'))])
#print('la taille de notre population :',femme_prop)
len_pop_F = len(df.loc[(df['sexe'] == 'Feminin')])
print('la proportion des femmes mari´ees qui n’ont pas fait de mastersni de doctorat %',(femme_prop*100)/len_pop_F)

homme_prop = femme_prop = len(df.loc[((df['etat.civil'] == 'Marie_epoux_absent') | (df['etat.civil'] == 'Marie_civil')) & ((df['education'] != 'Doctorate') & (df['education'] != 'Masters') & (df['sexe'] == 'Masculin'))])
#print('la taille de notre population :',homme_prop)
len_pop_M = len(df.loc[(df['sexe'] == 'Masculin')])
print('la proportion des homme mari´ees qui n’ont pas fait de mastersni de doctorat %',(homme_prop*100)/len_pop_M)

la proportion des femmes mari´ees qui n’ont pas fait de mastersni de doctorat % 15.940952557794077
la proportion des homme mari´ees qui n’ont pas fait de mastersni de doctorat % 56.74162459843965


In [ ]:
#11) Les ressortissants de quel pays ont les salaires les plus ´elev´es uniquement
#dans notre ´echantillon?
contry_max = (df.loc[df['salaire'] == '>50K'])
contry_max = contry_max.groupby(['pays.natal']).size().sort_values(ascending = False)
print(contry_max)


pays.natal
United-States         7171
?                      146
Philippines             61
Germany                 44
India                   40
Canada                  39
Mexico                  33
England                 30
Italy                   25
Cuba                    25
Japan                   24
China                   20
Taiwan                  20
Iran                    18
South                   16
Poland                  12
France                  12
Puerto-Rico             12
Jamaica                 10
El-Salvador              9
Greece                   8
Cambodia                 7
Yugoslavia               6
Hong                     6
Ireland                  5
Vietnam                  5
Ecuador                  4
Haiti                    4
Portugal                 4
Scotland                 3
Thailand                 3
Hungary                  3
Guatemala                3
Laos                     2
Dominican-Republic       2
Peru                     2
Trinadad&Tobago  

In [ ]:
#réponce 12):Quelle race a les salaires les plus ´elev´es uniquement dans notre ´echantillon?
race_max = (df.loc[df['salaire'] == '>50K'])
race_max = race_max.groupby(['race']).size().sort_values(ascending = False)
print(race_max)

race
Blanc          7117
Noir            387
Asiatique       276
Ameriendien      36
Autre            25
dtype: int64


In [ ]:
###########################################6.5#####################################################
#1)Est-ce qu’on peut dire que la moyenne d’heures travaill´ees par semaine est
#autour de 40h/semaine ? Est-ce que l’affirmer serait significatif ?
from scipy.stats import ttest_1samp 
import numpy as np 
heur_par_semain = df['heures.par.semaine'].values
test, pval = ttest_1samp(heur_par_semain, 40)
print('p-values',pval)
if pval < 0.05:
    print('rejeter l hypothèse nulle \n')
else:
    print('on accept lhypotèse nulle \n')
    

p-values 1.6483286793366825e-10
rejeter l hypothèse nulle 



In [19]:
#2)Est ce que les hommes ont en moyenne un salaire sup´erieur `a 50k compar´e
#aux femmes ? est-ce significatif ?
from scipy.stats import ttest_ind
from scipy import stats
import numpy as np 
df = pd.read_csv('salaires.csv')
df_M_F = df[['sexe','salaire']]
df_homme = df_M_F[(df_M_F['sexe'] == 'Masculin')]
df_femme = df_M_F[(df_M_F['sexe'] == 'Feminin')]
df_homme = df_homme.replace(to_replace = ['>50K','<=50K'], value = [1,0])
df_femme = df_femme.replace(to_replace = ['>50K','<=50K'], value = [1,0])

df_homme = df_homme['salaire']
print(df_homme)
df_femme = df_femme['salaire']
print(df_femme)
homme_std = np.std(df_homme)
print('homme std =',homme_std)
femme_std = np.std(df_femme)
print('femme_std =',femme_std)

ttest, pval = ttest_ind(df_homme, df_femme, equal_var=True)
print('p-value',pval)
if pval < 0.05:
    print('rejeter l hypothèse nulle \n')
else:
    print('on accept lhypotèse nulle \n')


  
   

6        0
9        1
11       1
13       1
14       1
        ..
32554    0
32555    1
32556    0
32558    1
32560    0
Name: salaire, Length: 21790, dtype: int64
0        0
1        0
2        0
3        0
4        0
        ..
32547    1
32548    0
32550    0
32557    0
32559    0
Name: salaire, Length: 10771, dtype: int64
homme std = 0.46071870187301667
femme_std = 0.31221622020176376
p-value 0.0
rejeter l hypothèse nulle 



In [ ]:
#3)Est ce qu’il y a un lien quelconque entre l’ˆage et le nombre d’heures travaill´e
#par semaine ? est-ce significatif ?
from scipy.stats import ttest_ind
from scipy import stats
import numpy as np 
df_heur_age = df[['age','heures.par.semaine']][0:16281]
#print(df_heur_age)
df_heur_age_2 = df[['age','heures.par.semaine']][16281:32561]
#print(df_heur_age_2)
df_heur1 = df_heur_age['heures.par.semaine']
df_heur2 = df_heur_age_2['heures.par.semaine']
moyenne_1 = df_heur1.mean()
print('la moyenne de la df_heur1 =',moyenne_1)
moyenne_2 = df_heur2.mean()
print('la moyenne de la df_heur2 =',moyenne_2)
ttest, pval = ttest_ind(df_heur1, df_heur2)
print('p-value', pval)
if pval < 0.05:
    print('rejeter l hypothèse nulle \n')
else:
    print('on accept lhypotèse nulle \n')



la moyenne de la df_heur1 = 40.933173637982925
la moyenne de la df_heur2 = 39.94170761670762
p-value 4.24686791127587e-13
rejeter l hypothèse nulle 

